In [1]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

In [2]:
llm = GooglePalm(google_api_key= api_key, temperature=0.7)

C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(llm('how to make vada paav with time calculation'))

**Ingredients**

For the vadas:

* 1 cup urad dal (black gram lentils)
* 1/2 cup idli rice
* 1/2 cup water
* 1 teaspoon cumin seeds
* 1/2 teaspoon salt
* 1/4 teaspoon turmeric powder
* Vegetable oil, for frying

For the paav:

* 4 large bread rolls, split in half
* 1 cup of potato curry
* 1/2 cup of green chutney
* 1/2 cup of tamarind chutney
* 1/4 cup of finely chopped onions
* 1/4 cup of finely chopped tomatoes
* 1/4 cup of finely chopped cilantro leaves

**Instructions**

**To make the vadas:**

1. Soak the urad dal and idli rice in water for 6-8 hours.
2. Drain the water and grind the lentils and rice together to a smooth paste. Add the cumin seeds, salt, and turmeric powder and mix well.
3. Heat some oil in a deep frying pan.
4. Take small balls of the vada batter and flatten them slightly.
5. Fry the vadas in the hot oil until golden brown on all sides.
6. Remove the vadas from the oil and drain on paper towels.

**To make the paav:**

1. Toast the bread rolls in a toaster oven o

## Now we are connecting with our database from MySQL

In [4]:
from langchain.utilities import SQLDatabase

In [5]:
db_user ='root'
db_password ='root'
db_host ='localhost'
db_name ='atliq_tshirts'

In [6]:
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)

In [7]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [8]:
from langchain_experimental.sql import SQLDatabaseChain

In [9]:
db_chain = SQLDatabaseChain(llm = llm, database = db, verbose = True)

C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\langchain_experimental\sql\base.py:75: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [10]:
q1 = db_chain.run('how many nike tshirts in small size')



> Entering new SQLDatabaseChain chain...
how many nike tshirts in small size
SQLQuery:SELECT count(*) FROM t_shirts WHERE brand = 'Nike' AND size = 'S'
SQLResult: [(3,)]
Answer:3
> Finished chain.


In [11]:
q1

'3'

In [12]:
q2 = db_chain.run('how much is the price of inventory for all small size tshirts?')



> Entering new SQLDatabaseChain chain...
how much is the price of inventory for all small size tshirts?
SQLQuery:SELECT SUM(price * stock_quantity) 
FROM t_shirts 
WHERE size = 'S'
SQLResult: [(Decimal('20689'),)]
Answer:20689
> Finished chain.


in the above example we could have like wrong answer i dont how did it manage to give correct answer bt we will giver it few more  defilcult so we will decide how we can improve it 

In [13]:
# q3 = db_chain.run('if we have to sell all the levis tshorts today with discount applied how much our store will genrate')

see in this question is dificult to address like its complex query and many it is using its own genrel knowledge like usually 
wehen get this question there should be coulumns like start date of discount and end date of discount we cant give discount all days so problem was that in our database we did not have any columns like startdate and enddate theen it uses oown knowledege without looking into database so we have to solve this problem suing concept of "few shot learning" lets see how we will use it

In [14]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

q3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('17178.550000'),)]
Answer:17178.55
> Finished chain.


In [15]:
q4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('17482'),)]
Answer:17482
> Finished chain.


In [16]:
q5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(70,), (17,), (15,)]
Answer:17
> Finished chain.


Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [17]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('102'),)]
Answer:102
> Finished chain.


### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [18]:
few_shots = [
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': q3} ,
    
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : q4},
    
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : q5
     }
]

In [19]:
exmpl = {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : str(q5)
     }

# Embedding

In [20]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [ ]:
embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
string = "How many white color Levi's shirt I have? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White',Result of the SQL query,q5"

In [ ]:
emb_exmpl = embeddings.embed_query(string)

In [ ]:
emb

In [ ]:
len(emb),emb[:5]

In [ ]:
to_vectorize = [''.join(i.values()) for i in few_shots]

In [ ]:
to_vectorize[2]

# VectorStore

we have to store this embeding somewhere so to store this embding we have vectorStore so we can store aur embeding into this

the job of VectorStore is to take input questions convert into embedings and find the similar looking few shots examples

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
# !pip install chromadb==0.4.15

In [ ]:
to_vectorize

In [ ]:
vectorstore = Chroma.from_texts(to_vectorize, 
                                embedding= embeddings, 
                                metadatas= few_shots)

In [ ]:
texts = ["Sample text 1", "Sample text 2", "Sample text 3"]
metadatas = [{"metadata_key": "metadata_value"} for _ in range(len(texts))]
ids = ['id1', 'id2', 'id3']

# Attempt to create the VectorStore using the simplified data
vectorstore = Chroma.from_texts(texts=texts, metadatas=metadatas, ids=ids)


becouse of some issue i am not able to use perform this code so i check there documentation and get to know that they have made some changes 

# Exploring ChromaDB 

In [ ]:
import chromadb
from chromadb.config import Settings

In [ ]:
import chromadb
client = chromadb.PersistentClient(path="/path/to/persist/directory")

In [ ]:
collection = client.create_collection('my_info')

In [ ]:
# collection.add(
#     documents=to_vectorize,
#     metadatas=[
#             {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
#      'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
# (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
# group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
#  """,
#      'SQLResult': "Result of the SQL query",
#      'Answer': q3} ,
    
#      {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
#       'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
#       'SQLResult': "Result of the SQL query",
#       'Answer' : q4},
    
#     {'Question': "How many white color Levi's shirt I have?",
#      'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
#      'SQLResult': "Result of the SQL query",
#      'Answer' : q5
#      }
#     ],
#     ids=['id1', 'id2' ,'id3']
# )


In [ ]:
collection.add(ids=['id1','id2','id3'],
              metadatas= few_shots,
              documents=to_vectorize)

In [ ]:
# collection.delete(ids=['id1','id2','id3'],)

In [ ]:
collection.peek()

In [ ]:
from langchain.prompts import SemanticSimilarityExampleSelector

In [ ]:
# example_selector = SemanticSimilarityExampleSelector(vectorstore= collection, k=2)

this wont work for use coz we have used diffrent approch 

In [ ]:
few_shots[1]

In [ ]:
example_1 = collection.query(query_texts=["If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?"],
                             n_results=1)

In [ ]:
example_1

## Now we have to give prompt to llm do not use any of your genral knowledge and make things up instead only use columns from the given table and from given database

In [ ]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [ ]:
print(_mysql_prompt)

In [ ]:
print(PROMPT_SUFFIX)

### so as you can see we langchain already provides sql prompt its really great

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [ ]:
from langchain.prompts import FewShotPromptTemplate

In [ ]:
few = FewShotPromptTemplate(example_selector=example_1, 
                      example_prompt=example_prompt,
                      suffix=PROMPT_SUFFIX, 
                      prefix=_mysql_prompt, input_variables=['input','table_info','top_k'] 
                     )